<a href="https://colab.research.google.com/github/varshanj-hub/Heart--Disease-QLoRA-RAG/blob/main/Fine_tuning_LLM_QLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -q -U transformers datasets peft accelerate bitsandbytes wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
import json
import wandb

# Initialize WandB
wandb.init(project="Heart-Disease-QLoRA-RAG")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: varshanj028 (varshanj028-srm-institute-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [4]:
model = prepare_model_for_kbit_training(model)

# LoRA Config (optimized for T4 GPU)
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, peft_config)

In [13]:
from google.colab import drive
drive.mount('/content/drive')

import json

file_path = '/content/drive/My Drive/Heart-disease-custom-LLM-data.json'

with open(file_path, 'r') as f:
    data = json.load(f)

# Optional: print or use the data
print(data)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[{'id': 1, 'question': 'What are the early signs of heart disease?', 'response': 'Early signs include fatigue, shortness of breath, chest discomfort (angina), and irregular heartbeat. Regular checkups are advised if you have risk factors like high cholesterol or diabetes.', 'category': 'Symptoms', 'severity': 'Medium', 'tags': ['early-warning', 'prevention']}, {'id': 2, 'question': 'My blood pressure is 150/100. Should I go to the ER?', 'response': 'This is considered hypertensive crisis. Seek emergency care if you also have severe headache, blurred vision, or confusion.', 'category': 'Emergency', 'severity': 'High', 'tags': ['hypertension', 'urgent']}, {'id': 3, 'question': 'Can heart disease be reversed with diet?', 'response': 'Some damage can be mitigated with a Mediterranean diet (rich in fish, olive oil, nuts). However, advanced cases require medical in

In [16]:
import json

# Load your original JSON
with open('/content/drive/My Drive/Heart-disease-custom-LLM-data.json', 'r') as f:
    data = json.load(f)

# Convert to DeepSeek-compatible JSONL
output_path = '/content/drive/My Drive/deepseek-finetune-data.jsonl'
with open(output_path, 'w') as outfile:
    for record in data:
        jsonl_entry = {
            "instruction": f"Answer the question about heart disease {record['category'].lower()} (Severity: {record['severity']}).",
            "input": record["question"],
            "output": record["response"],
            # Include metadata (optional)
            "metadata": {
                "category": record["category"],
                "tags": record["tags"]
            }
        }
        json.dump(jsonl_entry, outfile)
        outfile.write('\n')

print(f"Converted {len(data)} records to JSONL at: {output_path}")

Converted 200 records to JSONL at: /content/drive/My Drive/deepseek-finetune-data.jsonl


In [18]:
# Load dataset (replace with your JSONL path)
with open("/content/drive/My Drive/deepseek-finetune-data.jsonl", "r", encoding="utf-8") as f:
    samples = [json.loads(line) for line in f]

dataset = Dataset.from_list(samples)
train_test_split = dataset.train_test_split(test_size=0.2)

# Tokenization function
def tokenize_function(examples):
    # Format examples for instruction tuning
    texts = [
        f"<|system|>\n{examples['instruction'][i]}</s>\n"
        f"<|user|>\n{examples['input'][i]}</s>\n"
        f"<|assistant|>\n{examples['output'][i]}</s>"
        for i in range(len(examples['input']))
    ]

    # Tokenize
    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=512,  # Adjust based on your context needs
        padding="max_length",
        return_tensors="pt",
        add_special_tokens=True  # Handles <|system|>, <|user|> etc. automatically
    )

    # For causal LM training, labels = input_ids
    tokenized["labels"] = tokenized["input_ids"].clone()

    # Mask user/system tokens if needed (optional)
    # tokenized["labels"][tokenized["input_ids"] == tokenizer.system_token_id] = -100
    # tokenized["labels"][tokenized["input_ids"] == tokenizer.user_token_id] = -100

    return tokenized

In [19]:
tokenized_train = train_test_split["train"].map(tokenize_function, batched=True)
tokenized_eval = train_test_split["test"].map(tokenize_function, batched=True)

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [20]:
training_args = TrainingArguments(
    output_dir="./deepseek_qlora_finetuned",
    per_device_train_batch_size=2,   # Critical for T4 memory limits
    gradient_accumulation_steps=8,   # Simulates larger batch size
    num_train_epochs=30,
    learning_rate=2e-5,
    fp16=True,                       # Mixed precision
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    save_steps=100,
    report_to="wandb",
    optim="paged_adamw_8bit",        # Prevents OOM errors
)

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two va

Step,Training Loss,Validation Loss
50,1.000800,0.824995
100,0.582000,0.575590
150,0.547100,0.541850
200,0.518000,0.513463
250,0.500100,0.497011
300,0.493400,0.491563


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=300, training_loss=0.9264921061197917, metrics={'train_runtime': 2321.5153, 'train_samples_per_second': 2.068, 'train_steps_per_second': 0.129, 'total_flos': 2.2779059503104e+16, 'train_loss': 0.9264921061197917, 'epoch': 30.0})

In [22]:
# Save the fine-tuned model
model.save_pretrained("./fine_tuned_heart_disease_model")

# Save the tokenizer
tokenizer.save_pretrained("./fine_tuned_heart_disease_model")

# If you want to save the full model (not just adapters)
# This requires merging the LoRA adapters first
from peft import PeftModel

# Merge LoRA adapters with base model
merged_model = model.merge_and_unload()
merged_model.save_pretrained("./fine_tuned_heart_disease_model_merged")

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [23]:
#TO SAVE THE FINETUNED MODEL IN TH DRIVE

# Define save path in your Google Drive
save_dir = "/content/drive/My Drive/fine_tuned_heart_disease_model"

# Save the fine-tuned model
model.save_pretrained(save_dir)

# Save the tokenizer
tokenizer.save_pretrained(save_dir)


('/content/drive/My Drive/fine_tuned_heart_disease_model/tokenizer_config.json',
 '/content/drive/My Drive/fine_tuned_heart_disease_model/special_tokens_map.json',
 '/content/drive/My Drive/fine_tuned_heart_disease_model/chat_template.jinja',
 '/content/drive/My Drive/fine_tuned_heart_disease_model/tokenizer.json')